In [ ]:
import os
import itertools
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm
from thefuzz import fuzz

In [ ]:
default_colors = plt.rcParams["axes.prop_cycle"].by_key()["color"]

In [ ]:
registry = pd.read_csv("~/Box/dsi-core/11th-hour/idi-corporate-structure/20250630-0800-gleif-goldencopy-lei2-golden-copy.csv", low_memory=False)

In [ ]:
registry["LegalAddress-region"] = np.where(registry["Entity.LegalAddress.Region"].notna(), registry["Entity.LegalAddress.Region"], registry["Entity.LegalAddress.Country"])
registry["HeadquartersAddress-region"] = np.where(registry["Entity.HeadquartersAddress.Region"].notna(), registry["Entity.HeadquartersAddress.Region"], registry["Entity.HeadquartersAddress.Country"])

In [ ]:
us_or_canada = registry["Entity.LegalAddress.Country"].isin(["US", "CA"]) | registry["Entity.HeadquartersAddress.Country"].isin(["US", "CA"])

In [ ]:
sec2023 = pd.read_csv("../data/sec-2023.csv", dtype={"cik": str})

In [ ]:
good_matches = pd.DataFrame([
    {"LEI": "HQD377W2YR662HK5JX27", "CIK": "0000001800"},
    {"LEI": "9C1X8XOOTYY2FNYTVH06", "CIK": "0000040533"},
    {"LEI": "3C7474T6CDKPR9K6YT90", "CIK": "0000040545"},
    {"LEI": "549300XMP3KDCKJXIU47", "CIK": "0000062709"},
    {"LEI": "549300ZFEEJ2IP5VME73", "CIK": "0000093751"},
    {"LEI": "WHENKOULSSK7WUM60H03", "CIK": "0000106640"},
    {"LEI": "PBJHNT51T5V6G5UQFV42", "CIK": "0000351569"},
    {"LEI": "529900E95812SYWMCE44", "CIK": "0000723612"},
    {"LEI": "ORQTRC074CWLT3DKHT41", "CIK": "0000740260"},
    {"LEI": "549300J4ZRR1SZF4ES33", "CIK": "0000783325"},
    {"LEI": "549300IGLYTZUK3PVP70", "CIK": "0000783325"},
    {"LEI": "FU4LY2G4933NH2E1CP29", "CIK": "0000796343"},
    {"LEI": "549300D3L3G0R4U4VT04", "CIK": "0000820313"},
    {"LEI": "549300LRGRD8XC9BTL66", "CIK": "0000851310"},
    {"LEI": "549300HWF93HB8BDM648", "CIK": "0000883237"},
    {"LEI": "784F5XWPLTWKTBV3E584", "CIK": "0000886982"},
    {"LEI": "549300VFZ8XJ9NUPU221", "CIK": "0001001250"},
    {"LEI": "SQL3F6CKNNBM3SQGHX24", "CIK": "0001020569"},
    {"LEI": "2T3D6M0JSY48PSZI1Q41", "CIK": "0001039684"},
    {"LEI": "P16VZZCKTJ5IONL6XE37", "CIK": "0001042729"},
    {"LEI": "5493000C01ZX7D35SD85", "CIK": "0001067983"},
    {"LEI": "549300UJLWOIWFDGB318", "CIK": "0001070081"},
    {"LEI": "549300VSMO9KYQWDND94", "CIK": "0001094285"},
    {"LEI": "54930073EK7NHVJF3D24", "CIK": "0001102432"},
    {"LEI": "549300DV9GIB88LZ5P30", "CIK": "0001103982"},
    {"LEI": "CUMYEZJOAF02RYZ1JJ85", "CIK": "0001126328"},
    {"LEI": "9695005J16R56PCL8N38", "CIK": "0001140536"},
    {"LEI": "549300WHC56FF48KL350", "CIK": "0001140536"},
    {"LEI": "529900EVVV534W8R0T32", "CIK": "0001234006"},
    {"LEI": "549300IH2S17MAIUCM32", "CIK": "0001273441"},
    {"LEI": "549300NPYMLM4NHTOF27", "CIK": "0001274494"},
    {"LEI": "MTLVN9N7JE8MIBIJ1H73", "CIK": "0001276187"},
    {"LEI": "549300D4549QKWETZ406", "CIK": "0001336917"},
    {"LEI": "549300NDDBVLJS24LD07", "CIK": "0001370755"},
    {"LEI": "549300YG7Z2BC73TCX59", "CIK": "0001403161"},
    {"LEI": "HL3H1H2BGXWVG3BSWR90", "CIK": "0001413329"},
    {"LEI": "724500M9BY5293JDF951", "CIK": "0001413447"},
    {"LEI": "5493007CT6ATBZ2L6826", "CIK": "0001433660"},
    {"LEI": "549300VDIGTMXUNT7H71", "CIK": "0001551182"},
    {"LEI": "5493005LXSLNF92BF097", "CIK": "0001572334"},
    {"LEI": "529900E7PUVK6BC8PE80", "CIK": "0001590364"},
    {"LEI": "549300TUSNXS3BFFML48", "CIK": "0001627014"},
    {"LEI": "549300CMHPBEY6VPOT75", "CIK": "0001674910"},
    {"LEI": "ISRPG12PN4EIEOEMW547", "CIK": "0001735707"},
    {"LEI": "549300GZKULIZ0WOW665", "CIK": "0001744489"},
    {"LEI": "2549006ALJE6NRO2Q503", "CIK": "0001764046"},
    {"LEI": "5493008G0Y8W4B2BWL04", "CIK": "0001825079"},
    {"LEI": "549300XR4L1D80AK4W76", "CIK": "0001834584"},
])

In [ ]:
address_replacement = {
    "STREET": "ST",
    "AVENUE": "AVE",
    "BOULEVARD": "BLVD",
    "LANE": "LN",
    "DRIVE": "DR",
    "ROAD": "RD",
    "CRESCENT": "CRES",
    "PLACE": "PL",
    "TERRACE": "TER",
    "COURT": "CT",
    "CIRCLE": "CIR",
    "SQUARE": "SQ",
    "ALLEY": "ALY",
    "MOUNT": "MT",
    "HILL": "HL",
    "HILLS": "HLS",
    "ESTATE": "EST",
    "ESTATES": "ESTS",
    "GARDEN": "GDN",
    "GARDENS": "GDNS",
    "GREEN": "GRN",
    "GROVE": "GRV",
    "PARKWAY": "PKWY",
    "PARK": "PK",
    "PARKS": "PKS",
    "PARKLAND": "PKLD",
    "MARKET": "MKT",
    "HIGHWAY": "HWY",
    "TOLLWAY": "TLWY",
    "FLAT": "FLT",
    "SUITE": "STE",
    "TOWER": "TWR",
    "BUILDING": "BLDG",
    "BLOCK": "BLK",
    "APARTMENT": "APT",
    "FLOOR": "FLR",
    "INDUSTRIAL": "IND",
    "CENTER": "CTR",
    "COMPLEX": "CMPLX",
    "UNIVERSITY": "UNIV",
    "INSTITUTE": "INST",
    "PLAZA": "PLZ",
    "TRAIL": "TRL",
    "BRIDGE": "BRG",
    "EAST": "E",
    "WEST": "W",
    "SOUTH": "S",
    "NORTH": "N",
    "POINT": "PT",
    "PENTHOUSE": "PH",
    "SAINT": "ST",
    "SAINTS": "STS",
    "JUNCTION": "JCT",
    "CROSSING": "XING",
    "EXPRESSWAY": "EXPY",
    "FREEWAY": "FWY",
    "EXTENSION": "EXT",
    "MEADOWS": "MDWS",
    "FIELDS": "FLDS",
    "FIELD": "FLD",
    "WOODS": "WDS",
    "FOREST": "FRST",
    "ROOM": "RM",
    "FIRST": "1ST",
    "SECOND": "2ND",
    "THIRD": "3RD",
    "FOURTH": "4TH",
    "FIFTH": "5TH",
    "SIXTH": "6TH",
    "SEVENTH": "7TH",
    "EIGHTH": "8TH",
    "NINTH": "9TH",
    "TENTH": "10TH",
    "ELEVENTH": "11TH",
    "TWELVTH": "12TH",
    "ONE": "1",
    "TWO": "2",
    "THREE": "3",
    "FOUR": "4",
    "FIVE": "5",
    "SIX": "6",
    "SEVEN": "7",
    "EIGHT": "8",
    "NINE": "9",
    "TEN": "10",
    "ELEVEN": "11",
    "TWELVE": "12",
    "THIRTEEN": "13",
    "FOURTEEN": "14",
    "FIFTEEN": "15",
    "SIXTEEN": "16",
    "SEVENTEEN": "17",
    "EIGHTEEN": "18",
    "NINTEEN": "19",
    "TWENTY": "20",
}

def normalize_address(address):
    if not isinstance(address, str):
        return ""
    return " ".join(
        address_replacement.get(word, word)
        for word in address.upper().replace("POST OFFICE", "PO").replace("P.O", "PO").replace(".", " ").replace(",", " ").replace("-", " ").split()
        if word != "C/O"
    )

In [ ]:
us_codes = {
    "ALABAMA": "AL",
    "KENTUCKY": "KY",
    "OHIO": "OH",
    "ALASKA": "AK",
    "LOUISIANA": "LA",
    "OKLAHOMA": "OK",
    "ARIZONA": "AZ",
    "MAINE": "ME",
    "OREGON": "OR",
    "ARKANSAS": "AR",
    "MARYLAND": "MD",
    "PENNSYLVANIA": "PA",
    "AMERICAN SAMOA": "AS",
    "MASSACHUSETTS": "MA",
    "PUERTO RICO": "PR",
    "CALIFORNIA": "CA",
    "MICHIGAN": "MI",
    "RHODE ISLAND": "RI",
    "COLORADO": "CO",
    "MINNESOTA": "MN",
    "SOUTH CAROLINA": "SC",
    "CONNECTICUT": "CT",
    "MISSISSIPPI": "MS",
    "SOUTH DAKOTA": "SD",
    "DELAWARE": "DE",
    "MISSOURI": "MO",
    "TENNESSEE": "TN",
    "DISTRICT OF COLUMBIA": "DC",
    "MONTANA": "MT",
    "TEXAS": "TX",
    "FLORIDA": "FL",
    "NEBRASKA": "NE",
    "TRUST TERRITORIES": "TT",
    "GEORGIA": "GA",
    "NEVADA": "NV",
    "UTAH": "UT",
    "GUAM": "GU",
    "NEW HAMPSHIRE": "NH",
    "VERMONT": "VT",
    "HAWAII": "HI",
    "NEW JERSEY": "NJ",
    "VIRGINIA": "VA",
    "IDAHO": "ID",
    "NEW MEXICO": "NM",
    "VIRGIN ISLANDS": "VI",
    "ILLINOIS": "IL",
    "NEW YORK": "NY",
    "WASHINGTON": "WA",
    "INDIANA": "IN",
    "NORTH CAROLINA": "NC",
    "WEST VIRGINIA": "WV",
    "IOWA": "IA",
    "NORTH DAKOTA": "ND",
    "WISCONSIN": "WI",
    "KANSAS": "KS",
    "NORTHERN MARIANA ISLANDS": "MP",
    "WYOMING": "WY",
}
ca_codes = {
    "ALBERTA": "AB",
    "BRITISH COLUMBIA": "BC",
    "MANITOBA": "MB",
    "NEW BRUNSWICK": "NB",
    "NEWFOUNDLAND AND LABRADOR": "NL",
    "NORTHWEST TERRITORIES": "NT",
    "NOVA SCOTIA": "NS",
    "NUNAVUT": "NU",
    "ONTARIO": "ON",
    "PRINCE EDWARD ISLAND": "PE",
    "QUEBEC": "QC",
    "QUÉBEC": "QC",
    "SASKATCHEWAN": "SK",
    "YUKON": "YT",
}

def states_to_codes(x):
    if not isinstance(x, str):
        return ""
    y = " ".join(x.upper().replace(".", " ").replace(",", " ").split())
    if y in us_codes.values():
        return f"US-{y}"
    if y in ca_codes.values():
        return f"CA-{y}"
    z = us_codes.get(y)
    if z is not None:
        return f"US-{z}"
    z = ca_codes.get(y)
    if z is not None:
        return f"CA-{z}"
    if y == "CAYMAN ISLANDS":
        return "KY"
    assert False

In [ ]:
columns = {
    "LEI": "LEI",
    "Entity.LegalName": "name",
    "Entity.LegalAddress.FirstAddressLine": "legal_address",
    "Entity.LegalAddress.City": "legal_city",
    "LegalAddress-region": "legal_region",
    "Entity.LegalAddress.PostalCode": "legal_zip",
    "Entity.HeadquartersAddress.FirstAddressLine": "hq_address",
    "Entity.HeadquartersAddress.City": "hq_city",
    "HeadquartersAddress-region": "hq_region",
    "Entity.HeadquartersAddress.PostalCode": "hq_zip",
}
gleif = registry[us_or_canada][columns.keys()].rename(columns=columns)
gleif["name"] = gleif["name"].str.upper()
gleif["legal_address"] = gleif["legal_address"].apply(normalize_address)
gleif["legal_city"] = gleif["legal_city"].str.upper()
gleif["legal_region"] = gleif["legal_region"].str.upper()
gleif["legal_zip"] = gleif["legal_zip"].str.upper().fillna("")
gleif["hq_address"] = gleif["hq_address"].apply(normalize_address)
gleif["hq_city"] = gleif["hq_city"].str.upper()
gleif["hq_region"] = gleif["hq_region"].str.upper()
gleif["hq_zip"] = gleif["hq_zip"].str.upper().fillna("")
gleif

In [ ]:
sec = pd.DataFrame({
    "CIK": sec2023["cik"],
    "name": sec2023["name"].str.upper(),
    "address": sec2023["address"].apply(normalize_address),
    "city": sec2023["city"].apply(lambda x: " ".join(x.upper().replace(".", " ").replace(",", " ").split()) if isinstance(x, str) else ""),
    "state": sec2023["state"].apply(states_to_codes),
    "zip": sec2023["zip"].str.upper().fillna(""),
})
sec

In [ ]:
LEI = np.empty(len(gleif) * len(sec), dtype=object)
CIK = np.empty(len(gleif) * len(sec), dtype=object)
name = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
legal_address = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
legal_city = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
legal_state = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
legal_zip = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
hq_address = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
hq_city = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
hq_state = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
hq_zip = np.full(len(gleif) * len(sec), -1, dtype=np.int8)
is_us = np.full(len(gleif) * len(sec), -1, dtype=np.int8)

In [ ]:
for i, (grow, srow) in tqdm(enumerate(itertools.product(gleif.itertuples(), sec.itertuples())), total=len(gleif) * len(sec)):
    LEI[i] = grow.LEI
    CIK[i] = srow.CIK
    name[i] = fuzz.ratio(grow.name, srow.name)
    legal_address[i] = fuzz.ratio(grow.legal_address, srow.address)
    legal_city[i] = fuzz.ratio(grow.legal_city, srow.city)
    legal_state[i] = fuzz.ratio(grow.legal_region, srow.state)
    legal_zip[i] = len(os.path.commonprefix([grow.legal_zip, srow.zip]))
    hq_address[i] = fuzz.ratio(grow.hq_address, srow.address)
    hq_city[i] = fuzz.ratio(grow.hq_city, srow.city)
    hq_state[i] = fuzz.ratio(grow.hq_region, srow.state)
    hq_zip[i] = len(os.path.commonprefix([grow.hq_zip, srow.zip]))
    yes = srow.state.startswith("US-")
    is_us[i] = (1 if grow.legal_region.startswith("US-") and yes else 0) + (2 if grow.hq_region.startswith("US-") and yes else 0)

In [ ]:
table = pa.Table.from_arrays([
        pa.array(LEI),
        pa.array(CIK),
        pa.array(name),
        pa.array(legal_address),
        pa.array(legal_city),
        pa.array(legal_state),
        pa.array(legal_zip),
        pa.array(hq_address),
        pa.array(hq_city),
        pa.array(hq_state),
        pa.array(hq_zip),
        pa.array(is_us),
    ],
    names=["LEI", "CIK", "name", "legal_address", "legal_city", "legal_state", "legal_zip", "hq_address", "hq_city", "hq_state", "hq_zip", "is_us"],
)

In [ ]:
BATCH_SIZE = 10000000

writer = pq.ParquetWriter(
    os.path.expanduser("~/Box/dsi-core/11th-hour/idi-corporate-structure/comparisons.parquet"),
    table.schema,
    compression="gzip",
    compression_level=4,
)

for start in tqdm(range(0, len(table), BATCH_SIZE)):
    stop = min(len(table), start + BATCH_SIZE)
    for batch in table[start:stop].to_batches():
        writer.write_batch(batch)

writer.close()

In [ ]:
comparisons = pd.read_parquet("~/Box/dsi-core/11th-hour/idi-corporate-structure/comparisons.parquet")

In [ ]:
comparisons